<a href="https://colab.research.google.com/github/Mago7322/colab-amount-total/blob/main/%E9%87%91%E9%A1%8D%E9%9B%86%E8%A8%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#── ライブラリインストール＆インポート ────────────────────────────
!pip install -q pdfplumber openpyxl

import re
import pandas as pd
import pdfplumber
from google.colab import files

#── PDFファイルをアップロード ──────────────────────────────────────
uploaded = files.upload()               # ダイアログで請求一覧表.pdf を選択
pdf_path = next(iter(uploaded.keys()))  # アップロードした PDF のファイル名


Saving 請求一覧表.pdf to 請求一覧表.pdf


# 請求一覧表

In [ ]:
import camelot
import pandas as pd
from google.colab import files
import re

# 1. PDFファイルアップロード
uploaded = files.upload()
pdf_filename = list(uploaded.keys())[0]

# 2. CamelotでPDF全ページのテーブル抽出
tables = camelot.read_pdf(pdf_filename, pages='all', flavor='stream')

# 3. 各ページの行ごとに正規化された構造に分解
records = []
for table in tables:
    df = table.df
    if df.shape[0] < 4:
        continue  # データが不足している表はスキップ

    for i in range(3, df.shape[0]):
        row = df.iloc[i]

        # --- 個別取引先: 6桁数字を抽出（例：065678） ---
        match_indiv = re.search(r'\b\d{6}\b', row[2])
        個別取引先 = match_indiv.group(0) if match_indiv else ''

        # --- 伝票番号: 行3（ゼロ埋め済み） ---
        伝票番号 = row[3].strip().replace('\n', '')

        # --- 発注日, 納品日, 仕入計上日: 行4（3日付連続） ---
        発注日, 納品日, 仕入計上日 = None, None, None
        match_date = re.search(
            r'(\d{4}/\d{2}/\d{2})\s*(\d{4}/\d{2}/\d{2})?\s*(\d{4}/\d{2}/\d{2})?',
            row[4]
        )
        if match_date:
            発注日, 納品日, 仕入計上日 = match_date.groups()

        # --- 請求額・納品先: 行5を金額＋文字列に分割 ---
        match_amt = re.match(r'([\d,]+)?\s*(.+)?', row[5].strip())
        請求額 = match_amt.group(1) if match_amt else ''
        納品先 = match_amt.group(2) if match_amt else ''

        # --- 伝票種別・入力日: 行6 ---
        match_slip = re.match(r'(\S+)?\s*(\d{4}/\d{1,2}/\d{1,2})?', row[6].strip())
        伝票種別 = match_slip.group(1) if match_slip else ''
        入力日 = match_slip.group(2) if match_slip else ''

        # --- 追加 ---
        records.append({
            "No": row[0].strip(),
            "ブロック": row[1].strip(),
            "個別取引先": 個別取引先,
            "伝票番号": 伝票番号,
            "発注日": 発注日,
            "納品日": 納品日,
            "仕入計上日": 仕入計上日,
            "請求額": 請求額,
            "納品先": 納品先,
            "伝票種別": 伝票種別,
            "入力日": 入力日,
        })

# 4. DataFrame化と欠損補完
df_final = pd.DataFrame(records)
df_final = df_final.ffill()

# 5. 出力ファイル作成とダウンロード
output_path = "請求一覧_整形済み_v9.xlsx"
df_final.to_excel(output_path, index=False)
files.download(output_path)


Saving 請求一覧表.pdf to 請求一覧表 (12).pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# カーマ得意先管理表

In [ ]:
# --- PDFテキスト抽出 ---
lines = []
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text() or ''
        lines.extend(text.splitlines())

# --- ヘッダ検出 ---
for i, ln in enumerate(lines):
    if all(k in ln for k in ["No", "ブロック", "個別取引先", "入力日"]):
        header_idx = i
        break
else:
    raise RuntimeError("テーブルヘッダが見つかりませんでした")

# --- 明細データ抽出 ---
records = []
date_pat = r'\d{4}/\d{1,2}/\d{1,2}'
for ln in lines[header_idx+1:]:
    s = ln.strip()
    if not s or not re.match(r'^\d{6}', s):
        continue
    m0 = re.match(r'^(\d{6})\s+(\d{2})\s+(\d+)\s+(\d+)', s)
    if not m0:
        continue
    rec = {
        'No':         m0.group(1),
        'ブロック':   m0.group(2),
        '個別取引先': m0.group(3),
        '伝票番号':   m0.group(4),
    }
    its = list(re.finditer(date_pat, s))
    if len(its) < 4:
        continue
    rec['発注日']       = its[0].group()
    rec['納品日']       = its[1].group()
    rec['仕入計上日']   = its[2].group()
    rec['入力日']       = its[-1].group()
    mid = s[its[2].end():its[-1].start()].strip()
    m_doc = re.search(r'\d{2}仕入', mid)
    if m_doc:
        rec['伝票種別'] = m_doc.group()
        mid = mid[:m_doc.start()].strip()
    else:
        rec['伝票種別'] = ''
    m_amt = re.match(r'(\d{1,3}(?:,\d{3})*)', mid)
    if m_amt:
        rec['請求額'] = m_amt.group(1)
        mid = mid[m_amt.end():].strip()
    else:
        rec['請求額'] = ''
    rec['納品先'] = mid
    records.append(rec)

# --- DataFrame化 ---
df = pd.DataFrame(records)

# --- データクリーニング・補完（納品先強化） ---
def clean_store(val):
    if pd.isna(val): return ''
    s = str(val).replace('　',' ').strip()
    m = re.match(r'^(\d{4})\s*([^\d].*)$', s)
    if m:
        return s
    return ''
df['納品先'] = df['納品先'].map(clean_store)
df['納品先'] = df['納品先'].replace({'': pd.NA}).ffill()

def split_store(val):
    if pd.isna(val): return ('', '')
    s = str(val).replace('　',' ').strip()
    m = re.match(r'^(\d{4})\s*([^\d].*)$', s)
    if m:
        return m.group(1), m.group(2).strip()
    return '', val

df['カーマコード'], df['店名'] = zip(*df['納品先'].map(split_store))

df['請求額'] = (
    df['請求額']
      .str.replace(',', '', regex=False)
      .str.replace('▲', '-', regex=False)
      .astype(float)
)

# --- カーマコード・店名ごとに合計金額を集計 ---
agg = (
    df.groupby(['カーマコード', '店名'], as_index=False)['請求額']
      .sum()
      .rename(columns={'請求額': '合計金額'})
)
agg = agg[['カーマコード', '店名', '合計金額']]

# --- Excel出力 ---
agg.to_excel('カーマ得意先管理表.xlsx', index=False)

print('✅ カーマ得意先管理表.xlsx（納品先分割強化・カーマコード・店名・合計金額）を出力しました')


✅ カーマ得意先管理表.xlsx（納品先分割強化・カーマコード・店名・合計金額）を出力しました


# デバッグ

In [ ]:
for i, table in enumerate(tables):
    print(f"\n--- Page {i+1} ---")
    print(table.df.head(10))  # 最初の10行だけ表示



--- Page 1 ---
        0      1           2                                          3  \
0                                                                請求金額合計   
1          請求書番号  0\n0000001                                              
2      No   ブロック  個別取引先 伝票番号                            発注日\n納品日\n仕入計上日   
3  000001     01    0\n05678  000097202 2025/03/01 2025/03/112025/03/11   
4  000002     01    0\n05678  000097243 2025/03/04 2025/03/082025/03/08   
5  000003     01    0\n05678  000097486 2025/03/05 2025/03/112025/03/11   
6  000004     01    0\n05678  000097627 2025/03/11 2025/03/282025/03/28   
7  000005     01    0\n05678                       000097886 2025/03/14   
8  000006     01    0\n05678  000097620 2025/03/18 2025/03/192025/03/19   
9  000007     01    0\n05678  000098054 2025/03/22 2025/03/312025/03/31   

                   4       5         6  
0         83,449,127                    
1               税\n0                    
2            請求額 納品先    伝票種別       